## Step 1 

So the idea right now is that, we take the UK temprature database and remove some coastal points using those coastal points that we have removed we try to conduct imputation. So for that we need to identify the points by the coast and later remove a sample of them and try to conduct imputation on the sample that we have removed. 

### Raster Temprature Data to vector polygons 

In [ ]:
import rasterio
from shapely.geometry import Point, box
import geopandas as gpd
import numpy as np
import pandas as pd
from tqdm import tqdm

# Path to your GeoTIFF file
input_file = 'UK_temp2.tif'

# Open the GeoTIFF file
with rasterio.open(input_file) as src:
    # Read the entire raster data into memory
    data = src.read(1)
    
    # Get the nodata value
    nodata_value = src.nodata

    # Create a mask for valid data points (including zero values)
    mask = (data != nodata_value) & ~np.isnan(data)
    
    # Get the row and column indices of valid data points
    row_indices, col_indices = np.where(mask)
    
    # Get the pixel values of the valid data points
    values = data[mask]
    
    # Initialize lists to store coordinates, values, and bounds
    lons, lats, x_mins, y_mins, x_maxs, y_maxs = [], [], [], [], [], []
    
    # Get pixel size (resolution)
    pixel_size_x, pixel_size_y = src.res
    
    # Iterate with progress tracking
    for row, col in tqdm(zip(row_indices, col_indices), total=len(row_indices), desc="Processing"):
        lon, lat = src.xy(row, col)
        lons.append(lon)
        lats.append(lat)
        x_min, y_min = lon - pixel_size_x / 2, lat - pixel_size_y / 2
        x_max, y_max = lon + pixel_size_x / 2, lat + pixel_size_y / 2
        x_mins.append(x_min)
        y_mins.append(y_min)
        x_maxs.append(x_max)
        y_maxs.append(y_max)

# Create a DataFrame from the valid points
df = pd.DataFrame({
    'longitude': lons,
    'latitude': lats,
    'value': values,
    'x_min': x_mins,
    'y_min': y_mins,
    'x_max': x_maxs,
    'y_max': y_maxs
})

# Create a GeoDataFrame from the DataFrame
gdf = gpd.GeoDataFrame(df, geometry=[box(x_min, y_min, x_max, y_max) for x_min, y_min, x_max, y_max in zip(df.x_min, df.y_min, df.x_max, df.y_max)], crs=src.crs)

# Save the GeoDataFrame as a shapefile
output_shapefile = 'UK_temp2.shp'
gdf.to_file(output_shapefile)

print(f"Shapefile saved successfully: {output_shapefile}")


Highlighting Polygons that are coastal (Idea if a square touches 7 or more squares it is landbased point )

Read algorithm 

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# Function to extract points from a square's coordinates
def extract_points(row):
    """Extracts the four corner points from a row of the GeoDataFrame."""
    x_min, y_min, x_max, y_max = row['x_min'], row['y_min'], row['x_max'], row['y_max']
    return set([
        (x_min, y_min),
        (x_max, y_min),
        (x_min, y_max),
        (x_max, y_max)
    ])

# Function to classify a square based on its points
def classify_square(row, all_squares):
    """Classify a square as 'inland' or 'coastal'."""
    points = extract_points(row)
    matching_count = 0
    
    # Compare this square's points with all other squares
    for other_index, other_row in all_squares.iterrows():
        if row.name == other_index:
            continue
        
        other_points = extract_points(other_row)
        # Count how many points match
        if len(points.intersection(other_points)) > 0:
            matching_count += 1
        
        # Early exit if enough matches are found
        if matching_count >= 7:
            return 'inland'
    
    return 'coastal'

# Load your shapefile into a GeoDataFrame
input_shapefile = 'UK_temp2.shp'
gdf = gpd.read_file(input_shapefile)

# Classify each square
gdf['classification'] = gdf.apply(lambda row: classify_square(row, gdf), axis=1)

# Save the classified GeoDataFrame to a new shapefile
output_shapefile = 'classified_squares2.shp'
gdf.to_file(output_shapefile)

print(f"Classified shapefile saved successfully: {output_shapefile}")
